#### Reimplement Task 1

In [2]:
import PyPDF2

#Extracting
def extract_text_from_pdf(pdf_file):
    text = ""
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)  # Use PdfReader instead of PdfFileReader
        for page_num in range(len(reader.pages)):  # reader.pages gives a list of pages
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

book1 = extract_text_from_pdf(r"C:\Users\Admin\Downloads\#Semester 7\NLP - Natural Language Processing\Gr homework\group 4\NLP-group-4\Data\1. Harry Potter and the Philosopher's Stone.pdf")
book2 = extract_text_from_pdf(r"C:\Users\Admin\Downloads\#Semester 7\NLP - Natural Language Processing\Gr homework\group 4\NLP-group-4\Data\2. Harry Potter and the Chamber of Secrets.pdf")
book3 = extract_text_from_pdf(r"C:\Users\Admin\Downloads\#Semester 7\NLP - Natural Language Processing\Gr homework\group 4\NLP-group-4\Data\3. Harry Potter and the Prisoner of Azkaban.pdf")
book4 = extract_text_from_pdf(r"C:\Users\Admin\Downloads\#Semester 7\NLP - Natural Language Processing\Gr homework\group 4\NLP-group-4\Data\4. Harry Potter and the Goblet.pdf")

In [3]:
# Text Preprocessing:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

processed_text1 = preprocess_text(book1)
processed_text2 = preprocess_text(book2)
processed_text3 = preprocess_text(book3)
processed_text4 = preprocess_text(book4)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import time
from sklearn.feature_extraction.text import CountVectorizer

def ensure_string(text):
    if isinstance(text, list):
        return ' '.join(text)  

def build_term_document_matrix(documents):
    vectorizer = CountVectorizer() 
    X = vectorizer.fit_transform(documents)  
    return X, vectorizer.get_feature_names_out()  

documents = [
    ensure_string(processed_text1),
    ensure_string(processed_text2),
    ensure_string(processed_text3),
    ensure_string(processed_text4)
]

start_time = time.time()

term_document_matrix, terms = build_term_document_matrix(documents)

execution_time = time.time() - start_time


print("Term-Document Matrix (Raw Frequency):")
print(term_document_matrix.toarray()) 

print("\nTerms (Features):")
print(terms)  

print(f"\nExecution Time: {execution_time} seconds")

Term-Document Matrix (Raw Frequency):
[[ 1  1  1 ...  2  0  1]
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  3  0  0]
 [ 1  1  1 ... 11  1  3]]

Terms (Features):
['10' '100' '101' ... 'zooming' 'éclair' 'éclairs']

Execution Time: 0.1634221076965332 seconds


In [5]:
# Euclidean Distance
from sklearn.metrics.pairwise import euclidean_distances

euclidean_dist = euclidean_distances(term_document_matrix)
print("Euclidean Distance Matrix:\n", euclidean_dist)

Euclidean Distance Matrix:
 [[   0.          916.1129843  1463.21256146 3420.60564813]
 [ 916.1129843     0.         1134.279507   2952.26082859]
 [1463.21256146 1134.279507      0.         2456.55042692]
 [3420.60564813 2952.26082859 2456.55042692    0.        ]]


In [6]:
# Consine similarity
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(term_document_matrix)
print("Cosine Similarity Matrix:\n", cosine_sim)

Cosine Similarity Matrix:
 [[1.         0.94329679 0.9416178  0.93467331]
 [0.94329679 1.         0.95078986 0.95214857]
 [0.9416178  0.95078986 1.         0.94577085]
 [0.93467331 0.95214857 0.94577085 1.        ]]


In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

documents = [' '.join(processed_text1), 
             ' '.join(processed_text2), 
             ' '.join(processed_text3), 
             ' '.join(processed_text4)]

representative_words = ['wizard', 'magical', 'harry', 'spells', 'witch', 'journey']

# Create a CountVectorizer with the selected words
vectorizer = CountVectorizer(vocabulary = representative_words)
term_document_matrix = vectorizer.fit_transform(documents).toarray()

df = pd.DataFrame(term_document_matrix, columns = representative_words)
print("Term-Document Matrix for Selected Words:\n", df)

Term-Document Matrix for Selected Words:
    wizard  magical  harry  spells  witch  journey
0      42       11   1308       6     12       11
1      47       16   1633       9     16        5
2      39       31   2035       2     42        6
3      83      125   3134      24     37       10


In [8]:
# Compute cosine similarity
cosine_sim = cosine_similarity(term_document_matrix)
print("Cosine Similarity Matrix for Selected Words:\n", cosine_sim)

Cosine Similarity Matrix for Selected Words:
 [[1.         0.99997861 0.99980603 0.99946822]
 [0.99997861 1.         0.9998701  0.99954149]
 [0.99980603 0.9998701  1.         0.999609  ]
 [0.99946822 0.99954149 0.999609   1.        ]]


In [9]:
# Reduce dimensionality using Truncated SVD
n_components = 2
svd = TruncatedSVD(n_components=n_components)
reduced_embeddings = svd.fit_transform(term_document_matrix)

print("Reduced Embeddings:\n", reduced_embeddings)

Reduced Embeddings:
 [[1308.55565611  -23.64973942]
 [1633.60579063  -26.99254366]
 [2035.79150708  -26.59805943]
 [3137.64510112   41.17428047]]


#### Main task 2: Implement tf-idf to replace the raw frequency of the term-doc matrix in Question 1. Recompute the similarity between 4 Harry Potter books with cosine similarity metric. 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents).toarray()

# Chuyển thành DataFrame để dễ dàng hiển thị
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF Matrix:")
print(tfidf_df)

# Tính toán Cosine Similarity bằng TF-IDF
tfidf_cosine_sim = cosine_similarity(tfidf_matrix)
print("Cosine Similarity Matrix with TF-IDF:\n", tfidf_cosine_sim)


TF-IDF Matrix:
         10       100       101       102       103       104       105  \
0  0.000475  0.000475  0.000475  0.000475  0.000475  0.000475  0.000475   
1  0.000384  0.000384  0.000384  0.000384  0.000384  0.000384  0.000384   
2  0.000302  0.000302  0.000302  0.000302  0.000302  0.000302  0.000302   
3  0.000185  0.000185  0.000185  0.000185  0.000185  0.000185  0.000185   

        106       107       108  ...    zombie     zonko    zontal       zoo  \
0  0.000581  0.000000  0.000475  ...  0.001436  0.000000  0.000000  0.005026   
1  0.000469  0.000000  0.000384  ...  0.000000  0.000000  0.000000  0.001160   
2  0.000370  0.000000  0.000302  ...  0.000457  0.005024  0.000000  0.000000   
3  0.000000  0.000355  0.000185  ...  0.000000  0.000280  0.000355  0.000000   

   zoological      zoom    zoomed   zooming    éclair   éclairs  
0    0.000000  0.000718  0.000475  0.001163  0.000000  0.000718  
1    0.000000  0.000000  0.000768  0.000000  0.000000  0.000000  
2    0.000

The part combines both ways for easy comparison 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessed documents (from your previous code)
documents = [' '.join(processed_text1), 
             ' '.join(processed_text2), 
             ' '.join(processed_text3), 
             ' '.join(processed_text4)]

# 1. Raw Frequency (CountVectorizer)
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(documents).toarray()

# Calculate Cosine Similarity with Raw Frequency
cosine_sim_raw = cosine_similarity(count_matrix)

# Convert to DataFrame for better readability
count_df = pd.DataFrame(count_matrix, columns=count_vectorizer.get_feature_names_out())

print("Term-Document Matrix with Raw Frequency:")
print(count_df)
print("\nCosine Similarity Matrix with Raw Frequency:")
print(pd.DataFrame(cosine_sim_raw, index=[f'Book {i+1}' for i in range(4)], columns=[f'Book {i+1}' for i in range(4)]))


# 2. TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents).toarray()

# Calculate Cosine Similarity with TF-IDF
cosine_sim_tfidf = cosine_similarity(tfidf_matrix)

# Convert to DataFrame for better readability
tfidf_df = pd.DataFrame(tfidf_matrix, columns=tfidf_vectorizer.get_feature_names_out())

print("\nTF-IDF Matrix:")
print(tfidf_df)
print("\nCosine Similarity Matrix with TF-IDF:")
print(pd.DataFrame(cosine_sim_tfidf, index=[f'Book {i+1}' for i in range(4)], columns=[f'Book {i+1}' for i in range(4)]))


Term-Document Matrix with Raw Frequency:
   10  100  101  102  103  104  105  106  107  108  ...  zombie  zonko  \
0   1    1    1    1    1    1    1    1    0    1  ...       2      0   
1   1    1    1    1    1    1    1    1    0    1  ...       0      0   
2   1    1    1    1    1    1    1    1    0    1  ...       1     11   
3   1    1    1    1    1    1    1    0    1    1  ...       0      1   

   zontal  zoo  zoological  zoom  zoomed  zooming  éclair  éclairs  
0       0    7           0     1       1        2       0        1  
1       0    2           0     0       2        0       0        0  
2       0    0           0     0       9        3       0        0  
3       1    0           1     4       9       11       1        3  

[4 rows x 15458 columns]

Cosine Similarity Matrix with Raw Frequency:
          Book 1    Book 2    Book 3    Book 4
Book 1  1.000000  0.943297  0.941618  0.934673
Book 2  0.943297  1.000000  0.950790  0.952149
Book 3  0.941618  0.950790  1.

- Raw Frequency: This matrix is based on the frequency of the words in each document, but does not take into account the importance of the word in each document, resulting in common words that can blur the differences between books.
=> Tends to give higher similarity values for documents that share a lot of common words, even if those words are not particularly important or representative.
- TF-IDF:When using TF-IDF, words that appear more in one document but less in other documents will have a higher weight. This helps improve the distinction between books
=> Lowers the similarity for common words and focuses more on terms that are specific to a given document, resulting in a more meaningful similarity measure.